<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/figures/ch18_statistics_figures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 18 (Integration applications in statistics)

---

# About this code file:

### This notebook will reproduce the figures in this chapter, and illustrate the mathematical concepts explained in the book. The point of providing the code is not just for you to recreate the figures, but for you to modify, adapt, explore, and experiment with the code.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from scipy import stats
import sympy.stats

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Figure 18.1: Example distribution of heights

In [ ]:
# height parameters (in cm)
mean  = 175
stdev = 7

heightsDomain = np.linspace(140,210,1001)

# generate random heights
pdf = stats.norm.pdf(heightsDomain,loc=mean,scale=stdev) * (heightsDomain[1]-heightsDomain[0])

# plot the distribution
plt.figure(figsize=(4,4))
plt.plot(heightsDomain,pdf,color='k')
plt.gca().set(xlim=heightsDomain[[0,-1]],xlabel='Height (cm)',ylabel='Probability',title='Distribution of heights')

plt.tight_layout()
plt.savefig('stats_heightsDist.png')
plt.show()

# Figure 18.2: Analytic vs. empirical distributions

In [ ]:
_,axs = plt.subplots(1,2,figsize=(11,4))

# analytic
x = np.linspace(-5,5,401)
pdf = stats.norm.pdf(x) * (x[1]-x[0])
axs[0].plot(x,pdf,'k')
axs[0].set(xlim=x[[0,-1]],xlabel='$x$',ylabel='Probability density',title=r'$\bf{A}$)  Analytic distribution')

# empirical
axs[1].set(xlim=x[[0,-1]],xlabel='$x$',ylabel='Count',title=r'$\bf{B}$)  Empirical histogram')
axs[1].hist(np.random.randn(4001),bins='auto',color=[.7,.7,.7])

plt.tight_layout()
plt.savefig('stats_analyticEmpiricalpdf.png')
plt.show()

# Figures 18.3-4: Normal distributions

In [ ]:
x,sigma = sym.symbols('x,sigma')

px = 1/sym.sqrt(2*sym.pi*sigma**2) * sym.exp( -x**2/(2*sigma**2) )
px_l = sym.lambdify((x,sigma),px)

px

In [ ]:
# draw the pdf with some values of sigma
sigmas = [ .2, 1, 2 ]
colors = np.linspace(.1,.7,len(sigmas))
styles = [ ':','-','--' ]

xx = np.linspace(-4,4,1001)
dx = xx[1] - xx[0]

plt.figure(figsize=(10,4))

for idx in range(len(sigmas)):

  # evaluate the function
  pdf = px_l(xx,sigmas[idx]) #* dx # for normalization

  # numerical check that sum(pdf)==1
  sumPdf = np.sum(pdf)

  # and plot
  plt.plot(xx,pdf,color=np.full(3,colors[idx]),linestyle=styles[idx],label=r'$\sigma$ = %g, $\sum p(x)$ = %.2f' %(sigmas[idx],sumPdf))


plt.legend()
plt.gca().set(xlim=xx[[0,-1]],xlabel='x',ylabel='Probability')
plt.tight_layout()
plt.savefig('stats_normal1.png')
plt.show()

# Figure 18.5: F-distributions

In [ ]:
# df pairs
df_pairs = [ [2,10], [5,30], [10,100] ]

fvals = np.linspace(.01,5,500)
dx = fvals[1]-fvals[0]

plt.figure(figsize=(10,4))


for idx in range(len(df_pairs)):

  # extract the df params
  d1,d2 = df_pairs[idx][0],df_pairs[idx][1]

  # a sympy expression for the distribution
  F = sym.stats.FDistribution('F',d1,d2)
  Fpdf_expr = sym.stats.density(F)(x)

  # lambdify that
  Fpdf_l = sym.lambdify(x,Fpdf_expr)

  # numerically evaluate the pdf
  pdf = Fpdf_l(fvals) * dx
  sumPdf = np.sum(pdf)

  # and plot
  plt.plot(fvals,pdf,color=np.full(3,colors[idx]),linestyle=styles[idx],label=r'$d_1,d_2$ = %g,%s, $\sum p(x)$ = %.2f' %(d1,d2,sumPdf))


plt.legend()
plt.gca().set(xlim=fvals[[0,-1]],xlabel='F',ylabel='Probability')
plt.tight_layout()
plt.savefig('stats_Fpdf.png')
plt.show()

# Figure 18.6: Pareto distribution

In [ ]:
# draw the pdf with some values of sigma
alphas = [ .2, 1, 2 ]
colors = np.linspace(.1,.7,len(alphas))
styles = [ ':','-','--' ]

xx = np.linspace(.5,5,401)
xm = 1
dx = xx[1] - xx[0]

plt.figure(figsize=(10,4))

for idx in range(len(alphas)):

  # evaluate the function
  pdf = alphas[idx] / (xx**(alphas[idx]+1))
  pdf[xx<(xm+dx)] = 0
  pdf *= dx

  # numerical check of sum(pdf)
  sumPdf = np.sum(pdf)

  # and plot
  plt.plot(xx,pdf,color=np.full(3,colors[idx]),linestyle=styles[idx],label=r'$\alpha$ = %g, $\sum p(x)$ = %.2f' %(alphas[idx],sumPdf))


plt.legend()
plt.gca().set(xlim=xx[[0,-1]],xlabel='x',ylabel='Probability')
plt.tight_layout()
plt.savefig('stats_pareto.png')
plt.show()

# Figure 18.7: cdf from pdf

In [ ]:
# height parameters (in cm)
mean  = 175
stdev = 7
targetHeight = 176

heightsDomain = np.linspace(140,210,1001)

targidx = np.argmin(abs(heightsDomain-targetHeight))

# generate random heights
pdf = stats.norm.pdf(heightsDomain,loc=mean,scale=stdev) * (heightsDomain[1]-heightsDomain[0])
cdf = stats.norm.cdf(heightsDomain,loc=mean,scale=stdev)

# plot the histogram
_,axs = plt.subplots(2,1,figsize=(10,6))
axs[0].plot(heightsDomain,pdf,color='k')
axs[0].plot([targetHeight,targetHeight],[0,pdf[targidx]],'k--')
axs[0].fill_between(heightsDomain[:targidx],pdf[:targidx],color='k',alpha=.2)
axs[0].set(xlim=heightsDomain[[0,-1]],xlabel='Height (cm)',
           ylim=[-.00004,np.max(pdf)*1.05],ylabel='Probability density',title=r'$\bf{A}$)  Probability density of heights')

axs[1].plot(heightsDomain,cdf,color='k')
axs[1].plot([targetHeight,targetHeight],[0,cdf[targidx]],'--',color=[.7,.7,.7])
axs[1].plot(heightsDomain[[0,targidx]],[cdf[targidx],cdf[targidx]],'--',color=[.7,.7,.7])
axs[1].text(145,cdf[targidx]*1.08,r'$c(%s)=%.3f$' %(targetHeight,cdf[targidx]))
axs[1].set(xlim=heightsDomain[[0,-1]],xlabel='Height (cm)',
           ylim=[-.015,1.02],ylabel='Cumulative probability',title=r'$\bf{B}$ Cumulative distribution of heights')


plt.tight_layout()
plt.savefig('stats_cdfFromPdf.png')
plt.show()

# Figure 18.8: Triangular cdf

In [ ]:
a = 1
b = 3
d = 4

xx = np.linspace(a-1,d+1,901)

# piece 1
cdf = np.zeros(len(xx))

# piece 2
whereX = (xx>a) & (xx<=b)
cdf[whereX] = (xx[whereX]-a)**2 / ((d-a)*(b-a))

# piece 3
whereX = (xx>b) & (xx<d)
cdf[whereX] = 1 - ( (d-xx[whereX])**2 / ((d-a)*(d-b)) )

# piece 4
whereX = xx>=d
cdf[whereX] = np.ones(np.sum(whereX))


plt.figure(figsize=(10,4))
plt.plot(xx,cdf,'k',label=r'$c(x)$')
plt.plot(a,0,'ks',markersize=12,markerfacecolor='w',label='a')
plt.plot(b,cdf[np.argmin(abs(xx-b))],'ko',markersize=12,markerfacecolor=[.7,.7,.7],label='b')
plt.plot(d,1,'k^',markersize=12,markerfacecolor=[.3,.3,.3],label='d')

plt.legend()
plt.gca().set(xlim=xx[[0,-1]],xlabel='x',ylabel='Cumulative probability')

plt.tight_layout()
plt.savefig('stats_triangular.png')
plt.show()

# Figure 18.9: Laplace cdf

In [ ]:
x = sym.symbols('x')
xx = np.linspace(-4,4,501)
dx = xx[1]-xx[0]
bvals  = [ .5,1,2 ]
muvals = [ -1,0,.5 ]

_,axs = plt.subplots(2,1,figsize=(10,6))


for idx in range(len(bvals)):

  # a sympy expression for the distribution
  mu = muvals[idx]
  b  = bvals[idx]
  laplacian = sym.stats.Laplace('L',mu,b)


  # CDF and plot
  cdf_expr = sym.stats.cdf(laplacian)(x)
  cdf = [ cdf_expr.subs(x,xi) for xi in xx ]
  axs[0].plot(xx,cdf,color=np.full(3,colors[idx]),linestyle=styles[idx],label=r'$b=%g, \mu=%.2f$' %(b,mu))
  axs[0].axvline(mu,linewidth=1,color=np.full(3,colors[idx]),linestyle=styles[idx])

  # PDF and plot
  pdf_expr = sym.stats.density(laplacian)(x)
  pdf = [ pdf_expr.subs(x,xi)*dx for xi in xx ]
  pdfSum = np.sum(pdf)
  axs[1].plot(xx,pdf,color=np.full(3,colors[idx]),linestyle=styles[idx],label=r'$\sum p(x)=%.2f$' %pdfSum)
  axs[1].axvline(mu,linewidth=1,color=np.full(3,colors[idx]),linestyle=styles[idx])


axs[0].legend()
axs[1].legend()
axs[0].set(xlim=xx[[0,-1]],xlabel='$x$',ylabel='Cumulative probability',title=r'$\bf{A}$)  Laplace cdfs')
axs[1].set(xlim=xx[[0,-1]],xlabel='$x$',ylabel='Probability',title=r'$\bf{B}$)  Laplace pdfs')

plt.tight_layout()
plt.savefig('stats_Laplaces.png')
plt.show()

# Figure 18.10: Logistic cdf

In [ ]:
# parameters
mus = [ -2, 0, 3 ]
sigmas = [ 1, .5, .2 ]

xx = np.linspace(-5,5,401)

_,axs = plt.subplots(2,1,figsize=(10,6))
colors = np.linspace(.1,.7,len(mus))
styles = [ ':','-','--' ]

for idx in range(len(mus)):

  # parameters
  m = mus[idx]
  s = sigmas[idx]

  # get the cdf values
  cdf = stats.logistic.cdf(xx,loc=m,scale=s)
  pdf = stats.logistic.pdf(xx,loc=m,scale=s) * (xx[1]-xx[0])

  # and plot
  axs[0].plot(xx,cdf,color=np.full(3,colors[idx]),linestyle=styles[idx],label=r'$\mu = %g, \sigma = %g$' %(m,s))
  axs[0].axvline(m,linewidth=1,color=np.full(3,colors[idx]),linestyle=styles[idx])
  axs[1].plot(xx,pdf,color=np.full(3,colors[idx]),linestyle=styles[idx],label=r'$\mu = %g, \sigma = %g$' %(m,s))
  axs[1].axvline(m,linewidth=1,color=np.full(3,colors[idx]),linestyle=styles[idx])


axs[0].legend()
axs[1].legend()
axs[0].set(xlim=xx[[0,-1]],xlabel='$x$',ylabel='Cumulative probability',title=r'$\bf{A}$)  Logistic cdfs')
axs[1].set(xlim=xx[[0,-1]],xlabel='$x$',ylabel='Probability',title=r'$\bf{B}$)  Logistic pdfs')

plt.tight_layout()
plt.savefig('stats_logisticCdf.png')
plt.show()

# Figure 18.11: Distribution of IQ scores

In [ ]:
mu = 100
sigma = 15

iqs = np.linspace(mu-4*sigma,mu+4*sigma,201)
pdf = stats.norm.pdf(iqs,loc=mu,scale=sigma) * (iqs[1]-iqs[0])
cdf = stats.norm.cdf(iqs,loc=mu,scale=sigma) # used later

plt.figure(figsize=(4,4))
plt.plot(iqs,pdf,'k')
plt.gca().set(xlim=iqs[[0,-1]],xlabel='IQ score',ylabel='Probability density')

plt.tight_layout()
plt.savefig('stats_iqpdf.png')
plt.show()

In [ ]:
# the data
IQs = np.array([ 106,96,110,125,119,103,112,114,99,108,106,94,101,115,111 ] )

# formula parameters
n = len(IQs)
xbar = np.mean(IQs)

# z-value
zval = (xbar-mu) / (sigma/np.sqrt(n))
print(f'The average is {xbar:.3f} and the z-value is {zval:.3f}')

In [ ]:
# p(107-109) using FTC-2
z107 = (107-mu) / (sigma/np.sqrt(n))
z109 = (109-mu) / (sigma/np.sqrt(n))

pval = stats.norm.cdf(z109) - stats.norm.cdf(z107)
pval

In [ ]:
# note: you can also normalize the cdf instead of calculating z
print( stats.norm.cdf(z107) )
print( stats.norm.cdf(107,loc=100,scale=15/np.sqrt(n)) )

# Figure 18.12: Statistical inference using integrals

In [ ]:
zz = np.linspace(-4,4,305)
pdf = stats.norm.pdf(zz) * (zz[1]-zz[0])
cdf = stats.norm.cdf(zz)

_,axs = plt.subplots(2,1,figsize=(10,5))

axs[0].plot(zz,pdf,'k',label='pdf')
axs[0].axvline(zval,color='k',linestyle='--',label=f'z = {zval:.2f}')
axs[0].fill_between(zz[zz>=zval],pdf[zz>=zval],color='k',alpha=.2,label='Area to calculate')
axs[0].legend()
axs[0].set(xlim=zz[[0,-1]],ylim=[0,1.1*np.max(pdf)],ylabel='Probability density',title=r'$\bf{A}$)  Z-pdf with sample z-value')

axs[1].plot(zz,cdf,'k',label='cdf')
axs[1].axvline(zval,color='k',linestyle='--',label=f'z = {zval:.2f}')
axs[1].legend()
axs[1].set(xlim=zz[[0,-1]],xlabel='Standard z scores',ylabel='Cumulative prob.',title=r'$\bf{B}$)  Z-cdf with sample z-value')

plt.tight_layout()
plt.savefig('stats_iqZtest.png')
plt.show()

In [ ]:
pval = 1 - stats.norm.cdf(zval)
pval

# Figures 18.13-14: Random numbers from distributions

In [ ]:
sample_size = 5000

# generate the random data (use one of these)
data = stats.pareto.rvs(b=5,size=sample_size)
disttype = 'Pareto'

# data = stats.laplace.rvs(size=sample_size)
# disttype = 'Laplace'



# extract the histogram (empirical distribution)
heights,bins = np.histogram(data,bins=40)
binCenters = (bins[:-1]+bins[1:]) / 2

# estimate the pdf/cdf
pdfEst = heights / sample_size
cdfEst = np.cumsum(pdfEst)

# visualize
_,axs = plt.subplots(1,3,figsize=(14,4))

axs[0].plot(data,'ko',alpha=.3)
axs[0].set(xlabel='Data index',ylabel='Data value',title=r'$\bf{A}$)  Random %s data' %disttype)

axs[1].plot(binCenters,pdfEst,'k')
axs[1].set(xlim=bins[[0,-1]],xlabel='Data value',ylabel='Probability estimate',title=r'$\bf{B}$)  Histogram (estimated pdf)')

axs[2].plot(binCenters,cdfEst,'k')
axs[2].set(xlim=bins[[0,-1]],xlabel='Data value',ylabel='Cumulative probability',title=r'$\bf{C}$)  Estimated cdf')

plt.tight_layout()
plt.savefig(f'stats_rvs{disttype[0]}.png')
plt.show()

# Figure 18.16: Moments

In [ ]:
# moments in scipy
xx = np.linspace(-5,5,555)

_,axs = plt.subplots(1,3,figsize=(14,4))

### normal
pdf = stats.norm.pdf(xx)
moments = stats.norm.stats(moments='mvsk')

axs[0].plot(xx,pdf/np.max(pdf),'k')
for m in range(len(moments)):
  axs[0].text(0,.3-m/10,fr'$M_{{{m+1}}} = {{{moments[m]:.2f}}}$',ha='center',fontsize=16)

axs[0].set(xlim=xx[[0,-1]],xlabel='x',ylabel='Probability (norm.)',title=r'$\bf{A}$)  Normal pdf')



### logistic
pdf = stats.logistic.pdf(xx)
moments = stats.logistic.stats(moments='mvsk')

axs[2].plot(xx,pdf/np.max(pdf),'k')
for m in range(len(moments)):
  axs[2].text(0,.3-m/10,fr'$M_{{{m+1}}} = {{{moments[m]:.2f}}}$',ha='center',fontsize=16)

axs[2].set(xlim=xx[[0,-1]],xlabel='x',ylabel='Probability (norm.)',title=r'$\bf{C}$)  Logistic pdf')



### exponential Gaussian
xx = np.linspace(-3,9,555)
pdf = stats.exponnorm.pdf(xx,2)
moments = stats.exponnorm.stats(2,moments='mvsk')

axs[1].plot(xx,pdf/np.max(pdf),'k')
for m in range(len(moments)):
  axs[1].text(1.5,.3-m/10,fr'$M_{{{m+1}}} = {{{moments[m]:.2f}}}$',ha='center',fontsize=16)

axs[1].set(xlim=xx[[0,-1]],xlabel='x',ylabel='Probability (norm.)',title=r'$\bf{B}$)  Exponential Gaussian pdf')


plt.tight_layout()
plt.savefig('stats_pdfsWithMoments.png')
plt.show()

# Empirical moments in a small dataset

In [ ]:
data = np.arange(1,11)
print(f'First moment : {np.mean(data)}')
print(f'Second moment: {np.var(data)}')
print(f'Third moment : {stats.skew(data)}')
print(f'Fourth moment: {stats.kurtosis(data)}')